In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from functools import partial
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_page_load_timeout(60)


In [ ]:
url = 'https://www.quintoandar.com.br/comprar/imovel/sao-paulo-sp-brasil'
driver.get(url)
driver.implicitly_wait(10)

In [ ]:
enderecos = []
regioes = []
tipos_valores = []
outras_informacoes = []


In [ ]:
div = driver.find_element(By.CLASS_NAME,"sc-1y4z098-0.geAIvx")
time.sleep(2)
for i in range(30):
    enderecos.append([x.text for x in driver.find_elements(By.CSS_SELECTOR, 'span[data-testid="house-card-address"]')])
    regioes.append([x.text for x in driver.find_elements(By.CSS_SELECTOR, 'span[data-testid="house-card-region"]')])
    tipos_valores.append([x.text for x in  driver.find_elements(By.CLASS_NAME, 'sc-gsnTZi.iRsaMY.sc-crXcEl.jddosl.CozyTypography')])
    outras_informacoes.append(driver.find_elements(By.CSS_SELECTOR, 'small[data-testid="house-card-area"]'))
    driver.execute_script("arguments[0].scrollBy(0, 500);", div)


In [ ]:
outras_informacoes_clean = []
for i in outras_informacoes:
    for j in i:
        outras_informacoes_clean.append(j.text)

In [ ]:
enderecos = [item for sublist in enderecos for item in sublist]
regioes = [item for sublist in regioes for item in sublist]
tipos_valores = [item for sublist in tipos_valores for item in sublist]


In [ ]:
tipo = []
valores = []

for i in range(len(tipos_valores)):
    if tipos_valores[i].startswith("R$"):
        valores.append(tipos_valores[i])
    else:
        tipo.append(tipos_valores[i])

In [ ]:
metragem = []
n_quartos = []
n_vagas = []
for info in outras_informacoes_clean:
    splited_info  = info.split('•')
    metragem.append(int(splited_info[0].strip('m² ')))
    n_quartos.append(int(splited_info[1].strip(" quartos ")))
    n_vagas.append(int((splited_info[2].strip(" vagas "))))



In [ ]:
valores = [item.strip("R$ ").replace(".","")  for item in valores]
regioes = [item.strip("São Paulo").strip(',')  for item in regioes]


In [ ]:
real_state = {"tipo": tipo, "enderecos" : enderecos, "regioes" :  regioes, "valores" : valores, "metragem" : metragem, "n_quartos" : n_quartos, "n_vagas" : n_vagas}

In [ ]:
df = pd.DataFrame(real_state)


In [ ]:
df.to_csv("real_state.csv", index=False)

In [ ]:
df = pd.read_csv("real_state.csv")